# STAR PERFORMERS

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from bs4 import BeautifulSoup
import xmltodict
from pprint import pprint
import os
import csv
from api_keys import z_keys


In [2]:

# #config
# from api_keys import api_key
# from api_keys import g_Api

# # # Import API key
# # import api_keys

# # Incorporated citipy to determine city based on latitude and longitude
# from citipy import citipy

# # Output File (CSV)
# output_data_file = "output_data/cities.csv"

# # Range of latitudes and longitudes
# lat_range = (-90, 90)
# lng_range = (-180, 180)
# import openweathermapy.core as owm

In [3]:
ny18_df = pd.read_excel("/Users/JBagga/workspace/project_test/2018NYSales.xls")

In [4]:
len(ny18_df)

16369

In [5]:
ny18_df.describe()

,BOROUGH,BLOCK,LOT,ZIP CODE,RESIDENTIAL UNITS,COMMERCIAL UNITS,TOTAL UNITS,LAND SQUARE FEET,GROSS SQUARE FEET,YEAR BUILT,TAX CLASS AT TIME OF SALE,SALE PRICE
count,16369.0,16369.000000,16369.000000,16369.000000,16369.000000,16369.000000,16369.000000,16369.000000,1.636900e+04,16369.000000,16369.000000,1.636900e+04
mean,1.0,1134.104160,736.368990,10015.058281,1.837681,0.341438,2.305761,3799.704258,3.660774e+03,1702.245586,2.120716,3.068981e+06
std,0.0,526.607997,910.316535,385.473717,13.408085,4.091756,14.379706,12112.753126,2.819805e+04,652.533099,0.541500,2.760203e+07
min,1.0,2.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,1.000000,0.000000e+00
25%,1.0,798.000000,29.000000,10013.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,1910.000000,2.000000,2.307590e+05
50%,1.0,1171.000000,1002.000000,10022.000000,0.000000,0.000000,1.000000,0.000000,0.000000e+00,1931.000000,2.000000,8.150000e+05
75%,1.0,1469.000000,1185.000000,10028.000000,1.000000,0.000000,1.000000,891.000000,1.160000e+03,1973.000000,2.000000,1.850000e+06
max,1.0,2250.000000,9101.000000,10463.000000,597.000000,180.000000,601.000000,191502.000000,1.133548e+06,2018.000000,4.000000,2.397502e+09


In [6]:
ny18_df['ADDRESS'] = ny18_df['ADDRESS'].str.split(' ')
ny18_df


,BOROUGH,NEIGHBORHOOD,BUILDING CLASS CATEGORY,TAX CLASS AT PRESENT,BLOCK,LOT,EASE-MENT,BUILDING CLASS AT PRESENT,ADDRESS,APARTMENT NUMBER,...,RESIDENTIAL UNITS,COMMERCIAL UNITS,TOTAL UNITS,LAND SQUARE FEET,GROSS SQUARE FEET,YEAR BUILT,TAX CLASS AT TIME OF SALE,BUILDING CLASS AT TIME OF SALE,SALE PRICE,SALE DATE
0,1,ALPHABET CITY,01 ONE FAMILY DWELLINGS,1,390,61,,A4,"[189, EAST, 7TH, STREET]",,...,1,0,1,987,2183,1860,1,A4,4844809,2018-05-22
1,1,ALPHABET CITY,01 ONE FAMILY DWELLINGS,1,390,61,,A4,"[189, EAST, 7TH, STREET]",,...,1,0,1,987,2183,1860,1,A4,0,2018-05-23
2,1,ALPHABET CITY,01 ONE FAMILY DWELLINGS,1,400,19,,A4,"[526, EAST, 5TH, STREET]",,...,1,0,1,1883,5200,1900,1,A4,6100000,2018-12-03
3,1,ALPHABET CITY,02 TWO FAMILY DWELLINGS,1,390,35,,B1,"[113, AVENUE, C]",,...,2,0,2,1218,4764,1899,1,B1,0,2018-04-25
4,1,ALPHABET CITY,02 TWO FAMILY DWELLINGS,1,404,1,,B9,"[166, AVENUE, A]",,...,2,0,2,1510,4520,1900,1,B9,0,2018-11-29
5,1,ALPHABET CITY,02 TWO FAMILY DWELLINGS,1,404,1,,B9,"[166, AVENUE, A]",,...,2,0,2,1510,4520,1900,1,B9,0,2018-11-29
6,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2,379,37,,C7,"[133, AVENUE, D]",,...,20,2,22,3481,14588,1930,2,C7,8300000,2018-12-10
7,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2,385,2,,C7,"[21-23, AVENUE, B]",,...,12,4,16,4186,10588,1900,2,C7,10350000,2018-09-07
8,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2,389,18,,C1,"[200, EAST, 7TH, STREET]",,...,18,0,18,2271,10650,1910,2,C1,5000000,2018-09-05
9,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2A,390,54,,C3,"[203, EAST, 7TH, STREET]",,...,4,0,4,1950,5446,2001,2,C3,6250000,2018-05-09


In [7]:
ny18_df['ADDRESS'] = ny18_df['ADDRESS'].apply(lambda x: '+'.join(x))
# # ny18_df['ADDRESS'].str.join('+')
# ny18_df['ADDRESS'].str=ny18_df['ADDRESS'].str.replace(', ','+')
ny18_df

,BOROUGH,NEIGHBORHOOD,BUILDING CLASS CATEGORY,TAX CLASS AT PRESENT,BLOCK,LOT,EASE-MENT,BUILDING CLASS AT PRESENT,ADDRESS,APARTMENT NUMBER,...,RESIDENTIAL UNITS,COMMERCIAL UNITS,TOTAL UNITS,LAND SQUARE FEET,GROSS SQUARE FEET,YEAR BUILT,TAX CLASS AT TIME OF SALE,BUILDING CLASS AT TIME OF SALE,SALE PRICE,SALE DATE
0,1,ALPHABET CITY,01 ONE FAMILY DWELLINGS,1,390,61,,A4,189+EAST+7TH+STREET,,...,1,0,1,987,2183,1860,1,A4,4844809,2018-05-22
1,1,ALPHABET CITY,01 ONE FAMILY DWELLINGS,1,390,61,,A4,189+EAST+7TH+STREET,,...,1,0,1,987,2183,1860,1,A4,0,2018-05-23
2,1,ALPHABET CITY,01 ONE FAMILY DWELLINGS,1,400,19,,A4,526+EAST+5TH+STREET,,...,1,0,1,1883,5200,1900,1,A4,6100000,2018-12-03
3,1,ALPHABET CITY,02 TWO FAMILY DWELLINGS,1,390,35,,B1,113+AVENUE+C,,...,2,0,2,1218,4764,1899,1,B1,0,2018-04-25
4,1,ALPHABET CITY,02 TWO FAMILY DWELLINGS,1,404,1,,B9,166+AVENUE+A,,...,2,0,2,1510,4520,1900,1,B9,0,2018-11-29
5,1,ALPHABET CITY,02 TWO FAMILY DWELLINGS,1,404,1,,B9,166+AVENUE+A,,...,2,0,2,1510,4520,1900,1,B9,0,2018-11-29
6,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2,379,37,,C7,133+AVENUE+D,,...,20,2,22,3481,14588,1930,2,C7,8300000,2018-12-10
7,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2,385,2,,C7,21-23+AVENUE+B,,...,12,4,16,4186,10588,1900,2,C7,10350000,2018-09-07
8,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2,389,18,,C1,200+EAST+7TH+STREET,,...,18,0,18,2271,10650,1910,2,C1,5000000,2018-09-05
9,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2A,390,54,,C3,203+EAST+7TH+STREET,,...,4,0,4,1950,5446,2001,2,C3,6250000,2018-05-09


In [8]:
ny18_df_mod = pd.DataFrame(ny18_df)

In [9]:
len(ny18_df_mod)

16369

In [10]:
# Removing home prices less than 100K to avoid a bias after preliminary research on what these lower sale prices rep.
# like: Internal family sale transfer at $0, garage sale etc.

ny18_Update = ny18_df_mod[ny18_df_mod['SALE PRICE'] > 100000]

In [11]:
len(ny18_Update)

12406

In [12]:
ny18_Update.head()

,BOROUGH,NEIGHBORHOOD,BUILDING CLASS CATEGORY,TAX CLASS AT PRESENT,BLOCK,LOT,EASE-MENT,BUILDING CLASS AT PRESENT,ADDRESS,APARTMENT NUMBER,...,RESIDENTIAL UNITS,COMMERCIAL UNITS,TOTAL UNITS,LAND SQUARE FEET,GROSS SQUARE FEET,YEAR BUILT,TAX CLASS AT TIME OF SALE,BUILDING CLASS AT TIME OF SALE,SALE PRICE,SALE DATE
0,1,ALPHABET CITY,01 ONE FAMILY DWELLINGS,1,390,61,,A4,189+EAST+7TH+STREET,,...,1,0,1,987,2183,1860,1,A4,4844809,2018-05-22
2,1,ALPHABET CITY,01 ONE FAMILY DWELLINGS,1,400,19,,A4,526+EAST+5TH+STREET,,...,1,0,1,1883,5200,1900,1,A4,6100000,2018-12-03
6,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2,379,37,,C7,133+AVENUE+D,,...,20,2,22,3481,14588,1930,2,C7,8300000,2018-12-10
7,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2,385,2,,C7,21-23+AVENUE+B,,...,12,4,16,4186,10588,1900,2,C7,10350000,2018-09-07
8,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2,389,18,,C1,200+EAST+7TH+STREET,,...,18,0,18,2271,10650,1910,2,C1,5000000,2018-09-05


In [13]:
# from urllib.parse import urlparse
# ny18_Update['address_combo'] = urlparse.quote_plus(ny18_Update['ADDRESS'])

In [14]:
# urllib.parse.quote_plus

In [15]:
response = requests.get(f"https://www.zillow.com/webservice/GetDeepSearchResults.htm?zws-id={z_keys}&address=635+west+42+street&citystatezip=10036&rentzestimate=true")

In [16]:
response.text

'<?xml version="1.0" encoding="utf-8"?><SearchResults:searchresults xsi:schemaLocation="http://www.zillow.com/static/xsd/SearchResults.xsd https://www.zillowstatic.com/vstatic/091b98a/static/xsd/SearchResults.xsd" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xmlns:SearchResults="http://www.zillow.com/static/xsd/SearchResults.xsd"><request><address>635 west 42 street</address><citystatezip>10036</citystatezip></request><message><text>Request successfully processed</text><code>0</code></message><response><results><result><zpid>2129887786</zpid><links><homedetails>https://www.zillow.com/homedetails/635-W-42nd-Street-42a-42A-New-York-NY-10036/2129887786_zpid/</homedetails><mapthishome>http://www.zillow.com/homes/2129887786_zpid/</mapthishome><comparables>http://www.zillow.com/homes/comps/2129887786_zpid/</comparables></links><address><street>635 W 42nd Street 42a # 42A</street><zipcode>10036</zipcode><city>New York</city><state>NY</state><latitude>40.76126</latitude><longitude>-73

In [17]:
print(response)

<Response [200]>


In [18]:
#Using BeautifulSoup to convert XML text to pretty format
soup = BeautifulSoup(response.text)
print(soup.prettify())
# soup.find_all

<?xml version="1.0" encoding="utf-8"?>
<html>
 <body>
  <searchresults:searchresults xmlns:searchresults="http://www.zillow.com/static/xsd/SearchResults.xsd" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemalocation="http://www.zillow.com/static/xsd/SearchResults.xsd https://www.zillowstatic.com/vstatic/091b98a/static/xsd/SearchResults.xsd">
   <request>
    <address>
     635 west 42 street
    </address>
    <citystatezip>
     10036
    </citystatezip>
   </request>
   <message>
    <text>
     Request successfully processed
    </text>
    <code>
     0
    </code>
   </message>
   <response>
    <results>
     <result>
      <zpid>
       2129887786
      </zpid>
      <links>
       <homedetails>
        https://www.zillow.com/homedetails/635-W-42nd-Street-42a-42A-New-York-NY-10036/2129887786_zpid/
       </homedetails>
       <mapthishome>
        http://www.zillow.com/homes/2129887786_zpid/
       </mapthishome>
       <comparables>
        http://www.zillow.com/

In [19]:
# response['results']['result']['zpid']
#.keys() GIVES you the Current Level Keys in a Dictionary

doc = xmltodict.parse(response.text)['SearchResults:searchresults']['response']['results']['result']
# doc.values()
len(doc)

25

In [20]:
doc[0]['rentzestimate']['amount']['#text']
doc[4]['rentzestimate']['amount']['#text']

'5950'

In [21]:
pprint(doc)

[OrderedDict([('zpid', '2129887786'),
              ('links',
               OrderedDict([('homedetails',
                             'https://www.zillow.com/homedetails/635-W-42nd-Street-42a-42A-New-York-NY-10036/2129887786_zpid/'),
                            ('mapthishome',
                             'http://www.zillow.com/homes/2129887786_zpid/'),
                            ('comparables',
                             'http://www.zillow.com/homes/comps/2129887786_zpid/')])),
              ('address',
               OrderedDict([('street', '635 W 42nd Street 42a # 42A'),
                            ('zipcode', '10036'),
                            ('city', 'New York'),
                            ('state', 'NY'),
                            ('latitude', '40.76126'),
                            ('longitude', '-73.999164')])),
              ('FIPScounty', None),
              ('useCode', 'Apartment'),
              ('yearBuilt', '2006'),
              ('finishedSqFt', '1042'),
   

               OrderedDict([('region',
                             OrderedDict([('@name', "Hell's Kitchen"),
                                          ('@id', '778997'),
                                          ('@type', 'neighborhood'),
                                          ('links',
                                           OrderedDict([('overview',
                                                         "http://www.zillow.com/local-info/NY-New-York/Hell's-Kitchen/r_778997/"),
                                                        ('forSaleByOwner',
                                                         'http://www.zillow.com/hell%27s-kitchen-new-york-ny/fsbo/'),
                                                        ('forSale',
                                                         'http://www.zillow.com/hell%27s-kitchen-new-york-ny/')]))]))]))]),
 OrderedDict([('zpid', '244793225'),
              ('links',
               OrderedDict([('homedetails',
                  

                             OrderedDict([('@deprecated', 'true')])),
                            ('valueChange',
                             OrderedDict([('@duration', '30'),
                                          ('@currency', 'USD'),
                                          ('#text', '-130')])),
                            ('valuationRange',
                             OrderedDict([('low',
                                           OrderedDict([('@currency', 'USD'),
                                                        ('#text', '4657')])),
                                          ('high',
                                           OrderedDict([('@currency', 'USD'),
                                                        ('#text',
                                                         '6958')]))]))])),
              ('localRealEstate',
               OrderedDict([('region',
                             OrderedDict([('@name', "Hell's Kitchen"),
                            

                                          ('high',
                                           OrderedDict([('@currency', 'USD'),
                                                        ('#text',
                                                         '7913')]))]))])),
              ('localRealEstate',
               OrderedDict([('region',
                             OrderedDict([('@name', "Hell's Kitchen"),
                                          ('@id', '778997'),
                                          ('@type', 'neighborhood'),
                                          ('links',
                                           OrderedDict([('overview',
                                                         "http://www.zillow.com/local-info/NY-New-York/Hell's-Kitchen/r_778997/"),
                                                        ('forSaleByOwner',
                                                         'http://www.zillow.com/hell%27s-kitchen-new-york-ny/fsbo/'),
           

                                                         'http://www.zillow.com/hell%27s-kitchen-new-york-ny/')]))]))]))]),
 OrderedDict([('zpid', '79974820'),
              ('links',
               OrderedDict([('homedetails',
                             'https://www.zillow.com/homedetails/635-W-42nd-St-APT-2H-New-York-NY-10036/79974820_zpid/'),
                            ('graphsanddata',
                             'http://www.zillow.com/homedetails/635-W-42nd-St-APT-2H-New-York-NY-10036/79974820_zpid/#charts-and-data'),
                            ('mapthishome',
                             'http://www.zillow.com/homes/79974820_zpid/'),
                            ('comparables',
                             'http://www.zillow.com/homes/comps/79974820_zpid/')])),
              ('address',
               OrderedDict([('street', '635 W 42nd St APT 2H'),
                            ('zipcode', '10036'),
                            ('city', 'NEW YORK'),
                            

                                                         'http://www.zillow.com/hell%27s-kitchen-new-york-ny/')]))]))]))]),
 OrderedDict([('zpid', '79976919'),
              ('links',
               OrderedDict([('homedetails',
                             'https://www.zillow.com/homedetails/635-W-42nd-St-APT-44A-New-York-NY-10036/79976919_zpid/'),
                            ('graphsanddata',
                             'http://www.zillow.com/homedetails/635-W-42nd-St-APT-44A-New-York-NY-10036/79976919_zpid/#charts-and-data'),
                            ('mapthishome',
                             'http://www.zillow.com/homes/79976919_zpid/'),
                            ('comparables',
                             'http://www.zillow.com/homes/comps/79976919_zpid/')])),
              ('address',
               OrderedDict([('street', '635 W 42nd St APT 44A'),
                            ('zipcode', '10036'),
                            ('city', 'NEW YORK'),
                         

In [48]:
doc[1]['lastSoldPrice']['#text']

'1400000'

In [50]:
i=0
rent=[]
sold=[]
# zest=(doc[i]['rentzestimate']['amount']['#text'])
# zsold=(doc[i]['lastSoldPrice']['#text'])

while i<len(doc):
    if (doc[i].get('rentzestimate')) and not(doc[i].get('lastSoldPrice')):
        rent.append(doc[i]['rentzestimate']['amount']['#text'])
        sold.append('')
    elif not(doc[i].get('rentzestimate')) and (doc[i].get('lastSoldPrice')):
        sold.append(doc[i]['lastSoldPrice']['#text'])
        rent.append('')
    else:
        rent.append(doc[i]['rentzestimate']['amount']['#text'])
        sold.append(doc[i]['lastSoldPrice']['#text'])
    i+=1
print(f"Manhattan Property Sales Values = {sold}")

print(f"Manhattan Property Rental Values = {rent}")
len(rent)

Manhattan Property Sales Values = ['', '1400000', '1501918', '1185040', '1339763', '948000', '1552831', '1600000', '10', '995000', '', '', '', '', '1481553', '1100000', '', '10', '911333', '', '802890', '916425', '834965', '100', '1600000']
Manhattan Property Rental Values = ['6060', '5000', '6000', '', '5950', '3000', '6200', '5950', '6000', '4200', '5750', '5750', '6250', '3295', '6231', '4395', '7000', '3200', '3850', '5750', '3000', '4395', '3000', '3950', '5400']


25

In [ ]:
#### Need to loop through all the other cellls

In [ ]:
block =[]
lot=[]
sale_amt =[]
sale_date=[]

for row in ny18_Update['ADDRESS']:
    response_main = requests.get(f"https://www.zillow.com/webservice/GetDeepSearchResults.htm?zws-id={z_keys}&address={row}&citystatezip={ny18_Update['ZIP CODE']}&rentzestimate=true")
    block=ny18_Update['BLOCK']
    lot=ny18_Update['LOT']
    sale_amt=ny18_Update['SALE PRICE']
    sale_date=ny18_Update['SALE DATE']
                                 
    doc = xmltodict.parse(response.text)['SearchResults:searchresults']['response']['results']['result']                             
    time.sleep(.1)

In [ ]:
# response_main = requests.get("https://www.zillow.com/webservice/GetDeepSearchResults.htm?\
# zws-id={z_keys}&address=\
# west+42+street&citystatezip=10036&rentzestimate=true")